In [1]:
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import sqlite3
import time
import json
import random

load_dotenv()

True

In [2]:
fields = ["Registered_Address", "CEO", "Establishment_Year", "Number_Of_Employees", "Revenue_Size" ,
        "Website", "NAICS_Code", "SIC_Code", "Status"]

Define Class objects for Bing Search

In [3]:
class BingSearchAPI():

    def __init__(self) -> None:

        self.verbose = True
        self.n_webpages_to_scrape = 5

        # replace with list of keys once i run out
        self.subscription_key = os.getenv("BING_SEARCH_API_KEY")
        self.headers = {"Ocp-Apim-Subscription-Key": self.subscription_key}             
            
        self.search_url = "https://api.bing.microsoft.com/v7.0/search"

        self.params = {
            "q": "Insert Query here",
            "count": 20,  # Number of search results to return
            "offset": 0,  # The offset for pagination
            "mkt": "en-US",  # Region
            "safesearch": "Moderate"  # Safe search filter
        }

        self.search_results = {}
        self.webpages = {}
        self.request_timeout = 10

    def search(self, query: str) -> dict:
        self.webpages = {}
    
        if self.verbose: print("Conducting bing search for : ", query)
        
        self.params["q"] = query
        response = requests.get(self.search_url, headers=self.headers, params=self.params, timeout=self.request_timeout)

        try:
            response.raise_for_status()
        except requests.exceptions.HTTPError as e:
            print(f"Request failed: {e}")
            return None
        
        # handle api credit limit by switching to paid api key
        if response.status_code == 403:
            print("API key has reached its limit")
            return None
        
    
        
        self.search_results = response.json()
    
        return self.search_results
    

Read in list of firms from the Operncorporates

In [5]:
open_corporates_database = "../Opencorporates/Equifax Project Data - Sheet1.csv"
firms = pd.read_csv(open_corporates_database)
firms = firms.name.to_list()
print(len(firms))

10000


In [6]:
fields = ["Registered_Address", "CEO", "Establishment_Year", "Number_Of_Employees", "Revenue_Size" ,
        "Website", "NAICS_Code", "SIC_Code", "Status"]

Connect to sqlite table that stores web search results

In [7]:
conn = sqlite3.connect("firms_web_search_results.db")
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS firms_web_search_results (
               id INTEGER PRIMARY KEY AUTOINCREMENT,
               Firm_Name TEXT NOT NULL,
               Registered_Address TEXT,
               CEO TEXT,
               Establishment_Year TEXT,
               Number_Of_Employees TEXT,
               Revenue_Size TEXT,
               Website TEXT,
               NAICS_Code TEXT,
               SIC_Code TEXT,
               Status TEXT
               )
               ''')

Check how many firms are present

In [18]:
cursor.execute(''' SELECT COUNT(id) FROM firms_web_search_results ''')
print(cursor.fetchall())

[(1032,)]


Check for a specific firm

In [26]:
cursor.execute(f"SELECT * FROM firms_web_search_results WHERE id = 1017")
a = cursor.fetchall()
a

[(1017,
  '"K" LINE LOGISTICS (U.S.A.) INC.',
  '{"_type": "SearchResponse", "queryContext": {"originalQuery": "\\"K\\" LINE LOGISTICS (U.S.A.) INC. , Registered Address", "askUserForLocation": true}, "webPages": {"webSearchUrl": "https://www.bing.com/search?q=%22K%22+LINE+LOGISTICS+(U.S.A.)+INC.+%2c+Registered+Address", "totalEstimatedMatches": 4920000, "value": [{"id": "https://api.bing.microsoft.com/api/v7/#WebPages.0", "name": "\\"K\\u201d LINE LOGISTICS, LTD.", "url": "https://www.us.klinelogistics.com/", "isFamilyFriendly": true, "displayUrl": "https://www.us.klinelogistics.com", "snippet": "\\"K\\" Line Logistics, among KAWASAKI Group companies, has continued to grow as a core logistics enterprise. With our extensive global network of offices and companies at more than 70 locations worldwide, our managements and staff, experienced professionals, are in a good team spirit and committed to assisting our customers in tailoring and choosing the right logistic service plan that match

In [13]:
bing_searcher = BingSearchAPI()

### Loop to construct database

In [17]:
start_time = time.time()

for firm_name in firms:
    print("----- Debug: Now on firm: ", firm_name)

    # Check if firm already exists, insert row if it doesnt
    cursor.execute("SELECT id FROM firms_web_search_results WHERE Firm_Name = ?", (firm_name,))
    firm_row = cursor.fetchone()
    if firm_row is None:
        print("Inserting new firm:, ", firm_name)
        cursor.execute("INSERT INTO firms_web_search_results (Firm_Name) VALUES (?)", (firm_name,))
        firm_id = cursor.lastrowid 
    else:
        # Get the existing firm's id
        print("Found row for firm, ", firm_name)
        firm_id = firm_row[0]
    

    # Iterate through fields for the given firm
    for field in fields:

        # Check if the field value in the database is NULL
        cursor.execute(f"SELECT {field} FROM firms_web_search_results WHERE id = ? AND Firm_Name = ? AND {field} IS NOT NULL", (firm_id,firm_name,))
        if cursor.fetchone() is not None:
            print(f"Field '{field}' already has data for firm '{firm_name}', skipping.")
            continue

        query = f"{firm_name} , {field.replace('_', ' ')}"

        # print("---------- Debug: Now on field: ", field)

        # Get search results TODO: Handle search errors
        search_results = bing_searcher.search(query)
        
        if search_results is None:
            print("Search results are None")
            break

        json_combined_data = json.dumps(search_results)

        # Update cell value in database
        cursor.execute(f"""
                    UPDATE firms_web_search_results
                    SET {field} = ?
                    WHERE id = ? AND Firm_Name = ?
                    """, (json_combined_data, firm_id, firm_name))
        
        conn.commit()

     
cursor.close()
conn.close()

----- Debug: Now on firm:  " MROCK LLC "
Found row for firm,  " MROCK LLC "
Field 'Registered_Address' already has data for firm '" MROCK LLC "', skipping.
Field 'CEO' already has data for firm '" MROCK LLC "', skipping.
Field 'Establishment_Year' already has data for firm '" MROCK LLC "', skipping.
Field 'Number_Of_Employees' already has data for firm '" MROCK LLC "', skipping.
Field 'Revenue_Size' already has data for firm '" MROCK LLC "', skipping.
Field 'Website' already has data for firm '" MROCK LLC "', skipping.
Field 'NAICS_Code' already has data for firm '" MROCK LLC "', skipping.
Field 'SIC_Code' already has data for firm '" MROCK LLC "', skipping.
Field 'Status' already has data for firm '" MROCK LLC "', skipping.
----- Debug: Now on firm:  " OVER THE TOPP " INC
Found row for firm,  " OVER THE TOPP " INC
Field 'Registered_Address' already has data for firm '" OVER THE TOPP " INC', skipping.
Field 'CEO' already has data for firm '" OVER THE TOPP " INC', skipping.
Field 'Estab

KeyboardInterrupt: 

In [27]:
cursor.close()
conn.close()

In [40]:
b = json.loads(a[0][2])
b.keys()

dict_keys(['_type', 'queryContext', 'webPages', 'rankingResponse'])

In [42]:
b['webPages']

{'webSearchUrl': 'https://www.bing.com/search?q=%24+SAVE+INN%2c+INC.+%2c+Registered+Address',
 'totalEstimatedMatches': 2460000,
 'value': [{'id': 'https://api.bing.microsoft.com/api/v7/#WebPages.0',
   'name': '$ Save Inn, Inc. - AR, KS, MO, and MS - Bizapedia',
   'url': 'https://www.bizapedia.com/us/save-inn-inc.html',
   'isFamilyFriendly': True,
   'displayUrl': 'https://www.bizapedia.com/us/save-inn-inc.html',
   'snippet': 'There are 4 companies that go by the name of $ Save Inn, Inc. in Garden City KS, Jackson MS, Magnolia AR, Prescott AR, and Springfield MO ... Address: 302 Fleming, Suite 2, Garden City, KS 67846: Registered Agent: George P Naab: Filing Date: August 02, 1993: File Number: F00384072: Contact Us About The Company Profile For Save Inn, Inc.',
   'dateLastCrawled': '2024-07-01T03:13:00.0000000Z',
   'cachedPageUrl': 'http://cc.bingj.com/cache.aspx?q=%24+SAVE+INN%2c+INC.+%2c+Registered+Address&d=4714405670172541&mkt=en-US&setlang=en-US&w=3pCAvcBLBxtf3VIkIjils1TPpSa